In [ ]:
https://storage.yandexcloud.net/plates/tasks/1.csv

In [7]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm
import urllib.request
import os

#у каждого студента свой вариант
VARIANT = 100
VARIANT_URL = f'https://storage.yandexcloud.net/plates/tasks/{VARIANT}.csv'

#создаем папку data и скачиваем файл с ссылками на изображения
os.makedirs('data', exist_ok = True)
urllib.request.urlretrieve(VARIANT_URL, f'data/{VARIANT}.csv');

df = pd.read_csv(f'data/{VARIANT}.csv')

#создаем папку images и скачиваем туда изображения
os.makedirs('data/images', exist_ok = True)
for url in tqdm(list(df['url'])):
    name = url.split('/')[-1]
    path_img = f'data/images/{name}'
    if not os.path.exists(path_img):
        urllib.request.urlretrieve(url, path_img);

In [8]:
#ваша задача - прогнать все 2000 изображений через нейронную сеть и получить predict и уверенность в ответе
df.head(3)

,id,url
0,17297.jpg,https://storage.yandexcloud.net/plates/17297.jpg
1,64135.jpg,https://storage.yandexcloud.net/plates/64135.jpg
2,875.jpg,https://storage.yandexcloud.net/plates/875.jpg


In [9]:
#если у вас не получается самостоятельно прогнать, напишите мне, я вам выдам файл с предсказаниями
#но в этом случае вы не получите 2 балла за первый пункт задания
df = pd.read_csv(f'data/{VARIANT}_with_accuracy.csv')

#будем считать, что нейронка уверенна в ответе, если "accuracy" > 0.9995
df['is_good_prediction'] = (df['accuracy'] > 0.9995).astype(int)
df.head()

,id,url,predict,accuracy,is_good_prediction
0,17297.jpg,https://storage.yandexcloud.net/plates/17297.jpg,К071СО82,0.999935,1
1,64135.jpg,https://storage.yandexcloud.net/plates/64135.jpg,АМ479568,0.992788,0
2,875.jpg,https://storage.yandexcloud.net/plates/875.jpg,NaN,NaN,0
3,48905.jpg,https://storage.yandexcloud.net/plates/48905.jpg,В999РС21,0.999932,1
4,43329.jpg,https://storage.yandexcloud.net/plates/43329.jpg,NaN,NaN,0


In [10]:
df['is_good_prediction'].value_counts()

1    1562
0     438
Name: is_good_prediction, dtype: int64

В 1562 номерах наша нейрнонная сеть уверена.  
Для этих номеров в качестве ответа выдаем predict от нейронной сети.

В 438 номерах наша нейронная сеть не уверена. Отдадим эти изображения на разметку в краудсорсинг. 

In [11]:
from toloka_api import TolokaProcessingApi
import json

#запишем в переменную toloka_token наш токен для работы с API Яндекс.Толоки
toloka_token = json.loads(open("/var/www/nomeroff-net/romankucev/.ssh/tokens_toloka.txt").read())['IE']

toloka_api = TolokaProcessingApi(toloka_token)

In [12]:
#создадим проект из шаблона project_plates_19_10_20.json
project_plates = json.loads(open("project_templates/project_plates_19_10_20.json").read())
project_id = toloka_api.create_toloka_project(project_plates)

New project was created. New project id:  42941
https://toloka.yandex.ru/requester/project/42941


In [13]:
#выбираем изображения для разметки в Яндекс.Толоке
df_toloka = df[df['is_good_prediction'] == 0]

In [14]:
#переименовываем стоблцы
df_toloka = df_toloka.rename(columns={'url':'INPUT:image','predict':'INPUT:predict'})

In [15]:
#сохраняем файл с заданиями, который уже через UI загрузим в наш пул
os.makedirs('data/tsv', exist_ok = True)
df_toloka[['INPUT:image','INPUT:predict']].to_csv('data/tsv/df_toloka.tsv', sep='\t', index=None)

In [16]:
#для того, чтобы вам было проше, я разметил обучащие задания
urllib.request.urlretrieve('https://storage.yandexcloud.net/plates/obuch/obuch_tasks.tsv',
                           'data/tsv/obuch_tasks.tsv');

In [17]:
#для того, чтобы вам было проще, я составил вам немного контрольных заданий
urllib.request.urlretrieve('https://storage.yandexcloud.net/plates/control_tasks/control_tasks.tsv',
                           'data/tsv/control_tasks.tsv');

In [18]:
#запускаем пул, загружаем задания, размечаем.
#скачиваем разметку в файл res.tsv
res = pd.read_csv(f'data/tsv/res.tsv',sep = '\t')
res = res[~res['INPUT:image'].isnull()]

In [19]:
#посмотрим ответы исполнителей
for i in res['INPUT:image'].unique():
    print(i)
    tmp = res[res['INPUT:image'] == i]
    print(tmp['OUTPUT:output'].value_counts().to_string(),'\n');

https://storage.yandexcloud.net/plates/4096.jpg
Р819ОМ124    3 

https://storage.yandexcloud.net/plates/10999.jpg
Е253ОВ178    2
Е2530В178    1 

https://storage.yandexcloud.net/plates/control_tasks/1672_ее030226_ее03027_0.8331372737884521.jpg
ЕЕ030226    2 

https://storage.yandexcloud.net/plates/49761.jpg
М467ОР39    3 

https://storage.yandexcloud.net/plates/43212.jpg
Е133НН134    3 

https://storage.yandexcloud.net/plates/11094.jpg
4596АО21    3 

https://storage.yandexcloud.net/plates/45612.jpg
Х386СК78    2
К386СК78    1 

https://storage.yandexcloud.net/plates/22836.jpg
АТ276747    3 

https://storage.yandexcloud.net/plates/13899.jpg
О022РВ102    3 

https://storage.yandexcloud.net/plates/54982.jpg
М311ВХ39    3 

https://storage.yandexcloud.net/plates/17257.jpg
У157ЕС799    3 

https://storage.yandexcloud.net/plates/6027.jpg
Н657КС125    3 

https://storage.yandexcloud.net/plates/21061.jpg
4086ВН77    2
ВН408677    1 

https://storage.yandexcloud.net/plates/control_tasks/1599_в

У354СЕ56    3 

https://storage.yandexcloud.net/plates/30621.jpg
У324АО193    3 

https://storage.yandexcloud.net/plates/13080.jpg
2836АХ50    3 

https://storage.yandexcloud.net/plates/51813.jpg
ВС276577    3 

https://storage.yandexcloud.net/plates/56198.jpg
В257АХ799    3 

https://storage.yandexcloud.net/plates/control_tasks/1819_т984ма45_т745ооа0_0.7062792181968689.jpg
Т984МА45    2 

https://storage.yandexcloud.net/plates/56958.jpg
Н729РХ68    3 

https://storage.yandexcloud.net/plates/control_tasks/1824_а252аа152_4252аа62_0.995346188545227.jpg
А252АА152    3 

https://storage.yandexcloud.net/plates/54889.jpg
ЕЕ295150    3 

https://storage.yandexcloud.net/plates/48282.jpg
У727ТВ47    3 

https://storage.yandexcloud.net/plates/control_tasks/721_у941нн124_nan_nan.jpg
У941НН124    1 

https://storage.yandexcloud.net/plates/54792.jpg
М632МВ68    3 

https://storage.yandexcloud.net/plates/55831.jpg
Е058МА159    3 

https://storage.yandexcloud.net/plates/40659.jpg
А554ОС33    3 

http


https://storage.yandexcloud.net/plates/39842.jpg
ЕЕ43963    3 

https://storage.yandexcloud.net/plates/47162.jpg
С984СС99    3 

https://storage.yandexcloud.net/plates/27366.jpg
АЕ878605    2
В672СК59    1 

https://storage.yandexcloud.net/plates/40647.jpg
О015ВТ39    3 

https://storage.yandexcloud.net/plates/14390.jpg
М710РВ56    3 

https://storage.yandexcloud.net/plates/49860.jpg
С493ММ199    3 

https://storage.yandexcloud.net/plates/53293.jpg
НО982654    3 

https://storage.yandexcloud.net/plates/57147.jpg
ОО634377    3 

https://storage.yandexcloud.net/plates/51521.jpg
С885АТ**    3 

https://storage.yandexcloud.net/plates/44012.jpg
К835ЕВ05    3 

https://storage.yandexcloud.net/plates/7567.jpg
К063ОС199    3 

https://storage.yandexcloud.net/plates/42673.jpg
А888УН95    3 

https://storage.yandexcloud.net/plates/1999.jpg
АК419310    3 

https://storage.yandexcloud.net/plates/48804.jpg
Р501078    3 

https://storage.yandexcloud.net/plates/52661.jpg
У732КТ64    3 

https://stor


https://storage.yandexcloud.net/plates/43098.jpg
Т992СК799    3 

https://storage.yandexcloud.net/plates/63755.jpg
А531УР186    3 

https://storage.yandexcloud.net/plates/14457.jpg
А316МУ67    3 

https://storage.yandexcloud.net/plates/24476.jpg
Т973ТТ21    3 

https://storage.yandexcloud.net/plates/16172.jpg
ОХ71077    3 

https://storage.yandexcloud.net/plates/12861.jpg
С707ОК70    3 

https://storage.yandexcloud.net/plates/32784.jpg
АВ876703    3 

https://storage.yandexcloud.net/plates/32953.jpg
М854ВТ178    1
М854ВТ138    1
М854ВТ130    1 

https://storage.yandexcloud.net/plates/19723.jpg
Н357ЕУ55    3 

https://storage.yandexcloud.net/plates/1901.jpg
9779АЕ38    3 

https://storage.yandexcloud.net/plates/61710.jpg
3563АТ77    3 

https://storage.yandexcloud.net/plates/33502.jpg
ХЕ70577    3 

https://storage.yandexcloud.net/plates/18113.jpg
ВВ212716    3 

https://storage.yandexcloud.net/plates/43299.jpg
А260ОУ62    3 

https://storage.yandexcloud.net/plates/37663.jpg
3601ЕВ66  

In [20]:
#агрегируем с помощью majority vote
res_groupby = res[['INPUT:image','OUTPUT:output']].groupby(['INPUT:image']).agg(lambda x:x.value_counts().index[0])

In [21]:
#соединим ответы крауда с ответами нейронки
df = pd.merge(df,res_groupby,how='left',left_on='url',right_index = True)

In [24]:
#в итоге наши толокеры исправили 50 номеров 
df[(df['is_good_prediction'] == 0) & (df['predict'] != df['OUTPUT:output']) & (~df['predict'].isnull())]

,id,url,predict,accuracy,is_good_prediction,OUTPUT:output
33,14345.jpg,https://storage.yandexcloud.net/plates/14345.jpg,Е474ЕУ125,0.997695,0,Е674ЕУ125
42,45612.jpg,https://storage.yandexcloud.net/plates/45612.jpg,К386СК78,0.866302,0,Х386СК78
44,23200.jpg,https://storage.yandexcloud.net/plates/23200.jpg,В118СО134,0.988053,0,В118РО134
48,32127.jpg,https://storage.yandexcloud.net/plates/32127.jpg,А729СЕ124,0.659264,0,К729СЕ124
53,41890.jpg,https://storage.yandexcloud.net/plates/41890.jpg,3АТМ,0.717877,0,7083АТ77
171,50612.jpg,https://storage.yandexcloud.net/plates/50612.jpg,Н547ВК14,0.996701,0,М547ВК14
182,54982.jpg,https://storage.yandexcloud.net/plates/54982.jpg,М311В39,0.969475,0,М311ВХ39
203,59467.jpg,https://storage.yandexcloud.net/plates/59467.jpg,Н388ТУ45,0.951875,0,Н388ТУ48
232,51805.jpg,https://storage.yandexcloud.net/plates/51805.jpg,К039КХ79,0.891131,0,К039КХ799
283,62716.jpg,https://storage.yandexcloud.net/plates/62716.jpg,К462ВН136,0.958136,0,Н462ВН136


In [22]:
#запишем в answer наш итоговый ответ
df['answer'] = df['predict']
df.loc[df['is_good_prediction'] == 0,'answer'] = df['OUTPUT:output']

In [29]:
#сохраним нашу таблицу в файл answer.csv. Этот файл вы должны прислать мне 
df[['url','answer']].to_csv('data/tsv/answer.csv',index = None)

In [27]:
#этоот код вам не нужен, да и не запустится он у вас, так как ground_truth есть только у меня
#он необходим мне для проверки ваших решений
df = df[['url','answer','predict']]
df['id'] = df['url'].apply(lambda x: x.split('/')[-1])

In [28]:
#открываем ground_truth
ground_truth = pd.read_csv('../test_solution/threshold05.csv')[['id','plate']]
ground_truth['plate'] = ground_truth['plate'].str.upper()

In [29]:
#мерджим с вашим вариантом
data = pd.merge(df,ground_truth,how='left',left_on='id',right_on = 'id')
data.head()

,url,answer,predict,id,plate
0,https://storage.yandexcloud.net/plates/17297.jpg,К071СО82,К071СО82,17297.jpg,К071СО82
1,https://storage.yandexcloud.net/plates/64135.jpg,АМ479568,АМ479568,64135.jpg,АМ479568
2,https://storage.yandexcloud.net/plates/875.jpg,ВЕ94899,NaN,875.jpg,ВЕ94899
3,https://storage.yandexcloud.net/plates/48905.jpg,В999РС21,В999РС21,48905.jpg,В999РС21
4,https://storage.yandexcloud.net/plates/43329.jpg,М847ОО124,NaN,43329.jpg,М847ОО124


In [30]:
#считаем точность
accuracy = (data['answer'] == data['plate']).value_counts()
display(accuracy)
print(f'accuracy: {accuracy[True]/2000}')

True     1966
False      34
dtype: int64

accuracy: 0.983


In [31]:
#считаем точность без использования крауда
accuracy_nn = (data['predict'] == data['plate']).value_counts()
display(accuracy_nn)
print(f'accuracy: {accuracy_nn[True]/2000}')

True     1696
False     304
dtype: int64

accuracy: 0.848


In [43]:
data[data['answer'] != data['plate']]

,url,answer,predict,id,plate
57,https://storage.yandexcloud.net/plates/62258.jpg,Н547С***,NaN,62258.jpg,Н547СА69
61,https://storage.yandexcloud.net/plates/44611.jpg,4217КО2*,NaN,44611.jpg,4217КО23
122,https://storage.yandexcloud.net/plates/32953.jpg,М854ВТ130,NaN,32953.jpg,М854ВТ138
265,https://storage.yandexcloud.net/plates/12501.jpg,НА931453,NaN,12501.jpg,9314НА53
294,https://storage.yandexcloud.net/plates/33373.jpg,В709МН198,В709МН198,33373.jpg,В709ММ198
309,https://storage.yandexcloud.net/plates/56417.jpg,У553УЕ96,NaN,56417.jpg,У553КЕ96
366,https://storage.yandexcloud.net/plates/36249.jpg,Н967РН177,Н967РН177,36249.jpg,Р967РН177
376,https://storage.yandexcloud.net/plates/5628.jpg,В856МР82,Т856МР82,5628.jpg,Е749ВУ82
745,https://storage.yandexcloud.net/plates/63215.jpg,Т857АЕ193,Т857АЕ193,63215.jpg,Т857АЕ793
791,https://storage.yandexcloud.net/plates/57147.jpg,ОО634377,NaN,57147.jpg,ОО437763


In [44]:
data.head()

,url,answer,predict,id,plate
0,https://storage.yandexcloud.net/plates/17297.jpg,К071СО82,К071СО82,17297.jpg,К071СО82
1,https://storage.yandexcloud.net/plates/64135.jpg,АМ479568,АМ479568,64135.jpg,АМ479568
2,https://storage.yandexcloud.net/plates/875.jpg,ВЕ94899,NaN,875.jpg,ВЕ94899
3,https://storage.yandexcloud.net/plates/48905.jpg,В999РС21,В999РС21,48905.jpg,В999РС21
4,https://storage.yandexcloud.net/plates/43329.jpg,М847ОО124,NaN,43329.jpg,М847ОО124


In [47]:
#примеры, в которых ответ не сошелся с ground_truth
for url,anser,_,_, plate in data[data['answer'] != data['plate']].itertuples(index=False):
    print(url, plate, anser)

https://storage.yandexcloud.net/plates/62258.jpg Н547СА69 Н547С***
https://storage.yandexcloud.net/plates/44611.jpg 4217КО23 4217КО2*
https://storage.yandexcloud.net/plates/32953.jpg М854ВТ138 М854ВТ130
https://storage.yandexcloud.net/plates/12501.jpg 9314НА53 НА931453
https://storage.yandexcloud.net/plates/33373.jpg В709ММ198 В709МН198
https://storage.yandexcloud.net/plates/56417.jpg У553КЕ96 У553УЕ96
https://storage.yandexcloud.net/plates/36249.jpg Р967РН177 Н967РН177
https://storage.yandexcloud.net/plates/5628.jpg Е749ВУ82 В856МР82
https://storage.yandexcloud.net/plates/63215.jpg Т857АЕ793 Т857АЕ193
https://storage.yandexcloud.net/plates/57147.jpg ОО437763 ОО634377
https://storage.yandexcloud.net/plates/3763.jpg М653РО178 *653РО**
https://storage.yandexcloud.net/plates/38300.jpg Р218УС31 Р218У*31
https://storage.yandexcloud.net/plates/10404.jpg О964ТВ123 О694ТВ123
https://storage.yandexcloud.net/plates/16690.jpg М062ХС32 М062ХО32
https://storage.yandexcloud.net/plates/38926.jpg Н124

In [6]:
toloka_api.get_pool_completed_tasks(17472571)

,image,predict,output
0,https://storage.yandexcloud.net/plates/4096.jpg,Р819ОМ124,Р819ОМ124
1,https://storage.yandexcloud.net/plates/10999.jpg,None,Е253ОВ178
2,https://storage.yandexcloud.net/plates/control...,ЕЕ03027,ЕЕ030226
3,https://storage.yandexcloud.net/plates/49761.jpg,М467ОР39,М467ОР39
4,https://storage.yandexcloud.net/plates/43212.jpg,Е133НН134,Е133НН134
...,...,...,...
2845,https://storage.yandexcloud.net/plates/control...,К742НВ99,К742НВ99
2846,https://storage.yandexcloud.net/plates/28845.jpg,А626АА92,А626АА92
2847,https://storage.yandexcloud.net/plates/3566.jpg,Н072НУ159,Н072НУ159
2848,https://storage.yandexcloud.net/plates/56417.jpg,None,У5*


In [26]:
for var in range(51,96):
    print(f'https://storage.yandexcloud.net/plates/tasks/{var}.csv')

https://storage.yandexcloud.net/plates/tasks/51.csv
https://storage.yandexcloud.net/plates/tasks/52.csv
https://storage.yandexcloud.net/plates/tasks/53.csv
https://storage.yandexcloud.net/plates/tasks/54.csv
https://storage.yandexcloud.net/plates/tasks/55.csv
https://storage.yandexcloud.net/plates/tasks/56.csv
https://storage.yandexcloud.net/plates/tasks/57.csv
https://storage.yandexcloud.net/plates/tasks/58.csv
https://storage.yandexcloud.net/plates/tasks/59.csv
https://storage.yandexcloud.net/plates/tasks/60.csv
https://storage.yandexcloud.net/plates/tasks/61.csv
https://storage.yandexcloud.net/plates/tasks/62.csv
https://storage.yandexcloud.net/plates/tasks/63.csv
https://storage.yandexcloud.net/plates/tasks/64.csv
https://storage.yandexcloud.net/plates/tasks/65.csv
https://storage.yandexcloud.net/plates/tasks/66.csv
https://storage.yandexcloud.net/plates/tasks/67.csv
https://storage.yandexcloud.net/plates/tasks/68.csv
https://storage.yandexcloud.net/plates/tasks/69.csv
https://stor